# Copy-Generator Transformer

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from queue import PriorityQueue
import numpy as np
import torchtext
import tqdm.notebook as tqdm 
from torchnlp.metrics import get_moses_multi_bleu
from torchtext.data import Field, BucketIterator
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

import tensorflow as tf
import tensorflow_datasets as tfds
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

import linecache
import sys
import os
import re
import random
import time
import operator
import collections
from pprint import pprint

from models_and_trainers.base_transformer import TransformerModel, PositionalEncoding
from models_and_trainers.copy_gen_transformer import Transformer, TransformerDecoderLayer, TransformerDecoder
from models_and_trainers.retrieval import PyLuceneRetriever
# import beam_search
from utils.edit_tagger import build_matrix, single_step_edits, perform_edits, get_tags

from IPython.core.debugger import set_trace as tr
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
def super_print(filename):
    '''filename is the file where output will be written'''
    def wrap(func):
        '''func is the function you are "overriding", i.e. wrapping'''
        def wrapped_func(*args,**kwargs):
            '''*args and **kwargs are the arguments supplied 
            to the overridden function'''
            #use with statement to open, write to, and close the file safely
            with open(filename,'a') as outputfile:
                now = datetime.now()
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                outputfile.write("[{}] ".format(dt_string))
                outputfile.write(" ".join(str(x) for x in args))
                outputfile.write("\n")
            #now original function executed with its arguments as normal
            return func(*args,**kwargs)
        return wrapped_func
    return wrap

print = super_print('logs-copy-gen.txt')(print)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    torch.cuda.set_device(0) # choose GPU from nvidia-smi 
print("Using:", device)

Using: cuda


In [4]:
text = "create variable student_names with string 'foo bar baz'"

def string_split(s):
#     return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(_|\W)', s))) # this will chunk all code properly by plits strings with quotes
#     return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(\\\'.*?\\\'|\\\".*?\\\"|_|\W)', s))) # this keeps the strings intact
    return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(\\\'.*?\\\'|\\\".*?\\\"|\W)', s)))

print(string_split(text))

['create', 'variable', 'student_names', 'with', 'string', "'foo bar baz'"]


In [5]:
def corpus_to_array(src_fp, tgt_fp):
    lines = []
    with open(src_fp, "r") as src_file, open(tgt_fp, "r") as tgt_file:
        for src, tgt in zip(src_file, tgt_file):
            lines.append((src, tgt))
    return lines

In [6]:
def filter_corpus(data, max_seq_length=200, tokenizer=string_split):
    return [(src, tgt) for src, tgt in data if len(string_split(src)) <= max_seq_length and len(string_split(tgt)) <= max_seq_length]

In [7]:
def samples_to_dataset(samples):
    """
    Args:
        samples: [(src_string),(tgt_string)]
        src/tgt_tokenizer: a func that takes a string and returns an array of strings
    """
    examples = []
    TEXT_FIELD = Field(sequential=True, use_vocab=False, init_token='<sos>',eos_token='<eos>')
    
    for sample in samples:
        src_string, tgt_string = sample
        examples.append(torchtext.data.Example.fromdict({"src":src_string, "tgt":tgt_string}, 
                                        fields={"src":("src",TEXT_FIELD), "tgt":("tgt",TEXT_FIELD)}))
        
    dataset = torchtext.data.Dataset(examples,fields={"src":src_field, "tgt":tgt_field})
    return dataset

In [8]:
train_data = corpus_to_array("datasets/django_folds/django.fold1-10.train.src", "datasets/django_folds/django.fold1-10.train.tgt")
# data = corpus_to_array("datasets/all.desc", "datasets/all.code")
random.shuffle(train_data)
print("Max src length:", max([len(string_split(src)) for src, tgt in train_data]))
print("Max tgt length:", max([len(string_split(tgt)) for src, tgt in train_data]))

print("Full train dataset size:", len(train_data))
max_seq_length=50
train_data = filter_corpus(train_data, max_seq_length=max_seq_length, tokenizer=string_split)
train_data = [(" ".join(string_split(src))," ".join(string_split(tgt))) for src, tgt in train_data]
print("Limited train dataset size:", len(train_data))

Max src length: 557
Max tgt length: 527
Full train dataset size: 16924
Limited train dataset size: 16797


In [9]:
test_data = corpus_to_array("datasets/django_folds/django.fold1-10.test.src", "datasets/django_folds/django.fold1-10.test.tgt")
test_data = filter_corpus(test_data, max_seq_length=max_seq_length, tokenizer=string_split)
test_data = [(" ".join(string_split(src))," ".join(string_split(tgt))) for src, tgt in test_data]
print("Limited train dataset size:", len(test_data))

Limited train dataset size: 1874


In [10]:
stoi = {"<unk>":0, "<sos>":1, "<eos>":2, "<pad>":3, "<gen>":4}
max_vocab = 2000 - len(stoi)

all_toks = []
for (src, tgt) in train_data:
    all_toks += string_split(src)
    all_toks += string_split(tgt)

most_freq = collections.Counter(all_toks).most_common(max_vocab)

for tok, count in most_freq:
    stoi[tok] = len(stoi)
    
itos = [k for k,v in sorted(stoi.items(), key=lambda kv: kv[1])]

In [11]:
print(f"Vocabulary size: {len(stoi)}")

Vocabulary size: 2000


In [12]:
def encode_input(string, from_list=False):
    OOVs = []
    IDs = []
    if not from_list:
        words = string_split(string)
    else:
        words = string
    for word in words:
        try:
            id = stoi[word]
            IDs.append(id)
        except KeyError as e:
            # word is OOV
            IDs.append(len(stoi) + len(OOVs))
            OOVs.append(word)
    return IDs, OOVs

In [13]:
def encode_output(string, OOVs, from_list=False):
    IDs = []
    if from_list:
        words = string
    else:
        words = string_split(string)
    for word in words:
        try:
            id = stoi[word]
            IDs.append(id)
        except KeyError as e:
            # word is OOV
            try:
                IDs.append(len(stoi) + OOVs.index(word))
            except ValueError as e:
                IDs.append(stoi["<unk>"])
    return IDs

In [14]:
def decode(ids, OOVs):
    extended_itos = itos.copy()
    extended_itos += [OOV+"(COPY)" for OOV in OOVs]
    return " ".join([extended_itos[id] for id in ids if id<len(extended_itos)])

## Retrieval 
We want to find the most appropriate code to the input description. For this we use PyLucene to provide a BM25 search over the english descriptions.

In [15]:
train_data[0]

('if match is false or call to the method match . group with an argument integer 1 evaluates to boolean false ,',
 'if not match or not match . group ( 1 ) :')

In [16]:
retriever = PyLuceneRetriever()

src_data = [src for src,tgt in train_data]
retriever.add_multiple_docs(src_data)

In [17]:
doc_ranking = retriever.BM25_search("otherwise if common_prefix is None")
doc_ids = [x.doc for x in doc_ranking]
retrieved_samples = [train_data[i] for i in doc_ids]
print("10 best matched samples")
for doc in retrieved_samples[:10]:
    print(f"Description: {doc[0]}")
    print(f"Code       : {doc[1]}")
    print()

10 best matched samples
Description: otherwise if common_prefix is None ,
Code       : elif common_prefix is None :

Description: common_prefix is None .
Code       : common_prefix = None

Description: otherwise if prefix is not equal to common_prefix ,
Code       : elif prefix ! = common_prefix :



## Creating Edit steps
While there is a vocabulary for the shared english and code. The editing tokens also need to be converted to integers.

In [18]:
edit_stoi = {"K":0, "D":1, "R":2, "<pad>":3}
edit_itos = {0:"K", 1:"D", 2:"R", 3:"<pad>"}
max_insertions = 5

In [19]:
def encode_commands(commands):
    return [edit_stoi[command] for command in commands]

def decode_commands(commands):
    return [edit_itos[command] for command in commands]

### A single edit example
Let's make a complete sample from the dataset.
![alt text](./images/edit_transformer_diagram.png)

In [20]:
x, y = train_data[0]
doc_ranking = retriever.BM25_search(x)
top_2_doc = train_data[doc_ranking[2].doc] # get the second best document since the best doc is the same from the description
x_, y_ = top_2_doc

# convert to token array
x = string_split(x)
y = ["<sos>"] + string_split(y)
x_ = string_split(x_)
y_ = ["<sos>"] + string_split(y_)

print(y, y_)

dataset_edit_samples = []
while y_ != y:
    edit_steps = single_step_edits(y_, y)
    commands, insertions, replacements = edit_steps
    sample = {
        "encoder_input": x,
        "decoder_input": y_.copy(),
        "target_commands": commands,
        "target_insertions": insertions,
        "target_replacements": replacements,
        "code_target": y
    }
    dataset_edit_samples.append(sample)
    y_ = perform_edits(y_, edit_steps, gen_tok_id="<gen>")

pprint(dataset_edit_samples)

['<sos>', 'if', 'not', 'match', 'or', 'not', 'match', '.', 'group', '(', '1', ')', ':'] ['<sos>', 'if', 'not', 'self', '.', 'user_regex', '.', 'match', '(', 'user_part', ')', ':']
[{'code_target': ['<sos>',
                  'if',
                  'not',
                  'match',
                  'or',
                  'not',
                  'match',
                  '.',
                  'group',
                  '(',
                  '1',
                  ')',
                  ':'],
  'decoder_input': ['<sos>',
                    'if',
                    'not',
                    'self',
                    '.',
                    'user_regex',
                    '.',
                    'match',
                    '(',
                    'user_part',
                    ')',
                    ':'],
  'encoder_input': ['if',
                    'match',
                    'is',
                    'false',
                    'or',
                    'call',
  

### Making the dataset
The trick to modularizing models effectively is to make important functions that are necessary to them. `data2dataset()` is one suuch example. Taking in the dataset provided from the paper and converting it into the format needed to train our edit model.

In [21]:
OOV_stoi = {}
OOV_itos = {}
OOV_starter_count = 30000
OOV_count = OOV_starter_count

In [22]:
TEXT_FIELD = Field(sequential=True, use_vocab=False, unk_token=0, pad_token=3)
OOV_TEXT_FIELD = Field(sequential=True, use_vocab=False, pad_token=3)

def data2dataset(data, desc_rank=[1], token_insertions=1, single_step=False, max_insertions=5):
    global OOV_count
    TEXT_FIELD = Field(sequential=True, use_vocab=False, unk_token=0, pad_token=3)
    OOV_TEXT_FIELD = Field(sequential=True, use_vocab=False, pad_token=3)
    
    examples = []
    
    for (src, tgt) in tqdm.tqdm(data):
        
        for rank in desc_rank:
            doc_ranking = retriever.BM25_search(src)
            if len(doc_ranking) > rank:
                top_2_doc = train_data[doc_ranking[rank].doc]
                x_, y_ = top_2_doc
            else:
                x_, y_ = "", ""
            src_ids, OOVs = encode_input(src)
            decoder_input = encode_output(y_, OOVs)
            retrieved_code = decoder_input.copy()
            ground_truth_code = encode_output(tgt, OOVs)

            decoder_input = [stoi["<sos>"]] + decoder_input
            ground_truth_code = [stoi["<sos>"]] + ground_truth_code

            ran_once = False

            while decoder_input != ground_truth_code or not ran_once:
                ran_once = True
                edit_steps = single_step_edits(decoder_input, 
                                               ground_truth_code, 
                                               pad_token=stoi["<pad>"], 
                                               token_insertions=token_insertions,
                                               max_insertions=max_insertions)
                commands, target_insertions, target_replacements = edit_steps

                target_commands = encode_commands(commands)

                OOV_ids = []

                for OOV in OOVs:
                    try:
                        idx = OOV_stoi[OOV]
                        OOV_ids.append(idx)
                    except KeyError as e:
                        OOV_count += 1
                        OOV_stoi[OOV] = OOV_count
                        OOV_itos[OOV_count] = OOV
                        OOV_ids.append(OOV_count)


                if "<DELETE_ME>" in decoder_input:
                    print(decoder_input)

                example = torchtext.data.Example.fromdict({"encoder_input":src_ids, 
                                                           "ground_truth_code":ground_truth_code, 
                                                           "retrieved_code":retrieved_code,
                                                           "OOVs":OOV_ids, 
                                                           "decoder_input":decoder_input,
                                                           "target_commands":target_commands, 
                                                           "target_insertions": target_insertions,
                                                           "target_replacements":target_replacements}, 
                                                        fields={"encoder_input":("encoder_input",TEXT_FIELD), 
                                                                "ground_truth_code":("ground_truth_code",TEXT_FIELD),
                                                                "retrieved_code":("retrieved_code",TEXT_FIELD),
                                                                "OOVs":("OOVs", OOV_TEXT_FIELD), 
                                                                "decoder_input":("decoder_input",TEXT_FIELD),
                                                                "target_commands":("target_commands",TEXT_FIELD), 
                                                                "target_insertions": ("target_insertions",TEXT_FIELD),
                                                                "target_replacements":("target_replacements",TEXT_FIELD)})
                examples.append(example)
                decoder_input = perform_edits(decoder_input, edit_steps, gen_tok_id=stoi["<gen>"])
                if single_step:
                    break
    return examples

train_examples = data2dataset(train_data, token_insertions=-1, desc_rank=[1],max_insertions=max_insertions)
test_examples = data2dataset(test_data, desc_rank=[0], single_step=True) # single step here since we don't want to duplicate multi edit samples

In [23]:
print("OOV_itos size:", len(OOV_itos))

OOV_itos size: 6168


In [24]:
print("Num samples in train dataset:", len(train_examples))
print("Num samples in test dataset:", len(test_examples))

Num samples in train dataset: 21350
Num samples in test dataset: 1874


In [25]:
train_dataset = torchtext.data.Dataset(train_examples,fields={"encoder_input":TEXT_FIELD, 
                                                  "ground_truth_code":TEXT_FIELD, 
                                                  "retrieved_code":TEXT_FIELD,
                                                  "OOVs":OOV_TEXT_FIELD, 
                                                  "decoder_input":TEXT_FIELD, 
                                                  "target_commands":TEXT_FIELD, 
                                                  "target_insertions":TEXT_FIELD, 
                                                  "target_replacements":TEXT_FIELD})

val_dataset = torchtext.data.Dataset(test_examples,fields={"encoder_input":TEXT_FIELD, 
                                                  "ground_truth_code":TEXT_FIELD, 
                                                  "retrieved_code":TEXT_FIELD,
                                                  "OOVs":OOV_TEXT_FIELD, 
                                                  "decoder_input":TEXT_FIELD, 
                                                  "target_commands":TEXT_FIELD, 
                                                  "target_insertions":TEXT_FIELD, 
                                                  "target_replacements":TEXT_FIELD})
# train_dataset, val_dataset = dataset.split([0.9,0.1])
# train_dataset = val_dataset = dataset

In [26]:
def rmpad(arr):
    pad_id = stoi["<pad>"]
    return [x for x in arr if x != pad_id]

In [27]:
batch_size = 32

train_iterator = BucketIterator(
    train_dataset,
    batch_size = batch_size,
    repeat=True,
    shuffle=True,
    sort_key = lambda x: len(x.encoder_input)+len(x.decoder_input), # this doesn't seem to work, check it out later
    device = device)

# The iterator generates batches with padded length for sequences with similar sizes, a batch is [seq_length, batch_size]

for i, batch in enumerate(train_iterator):
    idx = 7
#     print([SRC_TEXT.vocab.itos[id] for id in batch.src.cpu().numpy()[:,idx]])
    OOVs = [OOV_itos[OOV] for OOV in batch.OOVs.cpu()[:,idx].tolist() if OOV != 3] # 3 is the <pad> token
    encoder_input = batch.encoder_input.cpu()[:,idx].tolist()
    decoder_input = batch.decoder_input.cpu()[:,idx].tolist()
    ground_truth_code = batch.ground_truth_code.cpu()[:,idx].tolist()
    target_commands = batch.target_commands.cpu()[:,idx].tolist()
    target_insertions = batch.target_insertions.cpu()[:,idx].tolist()
    target_replacements = batch.target_replacements.cpu()[:,idx].tolist()
    
    print("encoder_input    :",decode(rmpad(encoder_input), OOVs))
    print("decoder_input    :",decode(rmpad(decoder_input), OOVs))
    print("ground_truth_code:",decode(rmpad(ground_truth_code), OOVs))
    print("target_insertions  :", rmpad(target_insertions))
    print("target_replacements  :", target_replacements)
    print()
    break

encoder_input    : value under the ORDERING_FIELD_NAME(COPY) key of the form . fields is an instance of IntegerField class , created with 3 arguments :
decoder_input    : <sos> form . fields [ ORDERING_FIELD_NAME(COPY) ] = IntegerField ( label = _ ( <unk> ) , required = False )
ground_truth_code: <sos> form . fields [ ORDERING_FIELD_NAME(COPY) ] = IntegerField ( label = _ ( <unk> ) , initial = index + 1 , required = False )
target_insertions  : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0]
target_replacements  : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]



# The Model

In [28]:
class CopyModel(nn.Module):

    def __init__(self, vocab_size, embedding_size=512, dropout=0.5):
        super(CopyModel, self).__init__()
        self.model_type = 'Transformer'
        
        self.embedding_size = embedding_size
        self.pos_encoder = PositionalEncoding(embedding_size, dropout)
        self.src_encoder = nn.Embedding(vocab_size, embedding_size)
        self.tgt_encoder = nn.Embedding(vocab_size, embedding_size)
        
        self.transformer = Transformer(d_model=embedding_size, nhead=8, num_encoder_layers=4, num_decoder_layers=4, dim_feedforward=1024)
        self.replacement_decoder = nn.Linear(embedding_size, vocab_size)
        self.insertion_decoder = nn.Linear(embedding_size, max_insertions+1)
        self.command_decoder = nn.Linear(embedding_size, len(edit_stoi))
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.p_generator = nn.Linear(embedding_size,1)

        self.init_weights()
        self.tgt_mask = None

    def init_weights(self):
        initrange = 0.1
        self.src_encoder.weight.data.uniform_(-initrange, initrange)
        self.tgt_encoder.weight.data.uniform_(-initrange, initrange)
        
        self.replacement_decoder.bias.data.zero_()
        self.replacement_decoder.weight.data.uniform_(-initrange, initrange)
        
        self.insertion_decoder.bias.data.zero_()
        self.insertion_decoder.weight.data.uniform_(-initrange, initrange)
        
        self.command_decoder.bias.data.zero_()
        self.command_decoder.weight.data.uniform_(-initrange, initrange)
        
    def _generate_square_subsequent_mask(self, sz):
#         noise_e = 0.05 if self.training else 0.0 # this is code to add noise to the decoding process during training
        noise_e = 0.0 if self.training else 0.0
        noise_mask = (torch.rand(sz,sz) > noise_e).float()

        mask = (torch.triu(torch.ones(sz,sz))).transpose(0, 1)
        mask = torch.mul(mask, noise_mask)
        v = (torch.sum(mask, dim=-1) == 0).float()

        fix_mask = torch.zeros(sz,sz)
        fix_mask[:,0] = 1.0
        v = v.repeat(sz, 1).transpose(0,1)
        fix_mask = torch.mul(fix_mask,v)

        mask += fix_mask
        
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, tgt):
#         self.tgt_mask = self._generate_square_subsequent_mask(len(tgt)).to(self.device)
        

        src_emb = self.src_encoder(src) * math.sqrt(self.embedding_size)
        src_emb = self.pos_encoder(src_emb)
        
        tgt_emb = self.tgt_encoder(tgt) * math.sqrt(self.embedding_size)
        tgt_emb = self.pos_encoder(tgt_emb)
                
        output, atts = self.transformer(src_emb, tgt_emb)
        
        
        src_scat = src.transpose(0,1)
        src_scat = src_scat.unsqueeze(0)
        src_scat = torch.repeat_interleave(src_scat, tgt.shape[0], dim=0)
#         print("src_scat.shqape", src_scat.shape)
        
        p_gens = self.p_generator(output).sigmoid()
        atts = atts.transpose(0,1)
#         print("att.shqape", atts.shape)
        atts = atts * (1 - p_gens)
                
        target_replacements = self.replacement_decoder(output)
#         output[:,:,12:] = -np.inf
        target_replacements = target_replacements.softmax(-1)
        target_replacements = target_replacements * p_gens
        
        target_replacements = target_replacements.scatter_add_(2,src_scat,atts)
        
        target_insertions = self.insertion_decoder(output)
        
        target_commands = self.command_decoder(output)
        
        return target_commands, target_insertions, target_replacements.log()

In [29]:
stoi["','"]

597

In [30]:
vocab_size = len(itos) + max_seq_length

model = CopyModel(vocab_size).to(device) 
src = torch.randint(0, vocab_size, (3,2)).to(device)
tgt = torch.randint(0, vocab_size, (5,2)).to(device)

target_commands, target_insertions, target_replacements = model(src, tgt)
target_commands.shape, target_insertions.shape, target_replacements.shape

(torch.Size([5, 2, 4]), torch.Size([5, 2, 6]), torch.Size([5, 2, 2050]))

In [31]:
def nltk_bleu(refrence, prediction):
    """
    Implementation from ReCode
    and moses multi belu script sets BLEU to 0.0 if len(toks) < 4
    """
    ngram_weights = [0.25] * min(4, len(refrence))
    return sentence_bleu([refrence], prediction, weights=ngram_weights, 
                          smoothing_function=SmoothingFunction().method3)

In [32]:
def outputs2code(target_commands, target_insertions, target_replacements, decoder_input):
    batch_size = target_commands.shape[1]
    
    edited_code_samples = []
    for i in range(batch_size):
        code_to_edit = rmpad(decoder_input[:,i].tolist())
        code_length = len(code_to_edit)
        
        sample_commands = target_commands[:code_length,i].view(-1).tolist()
#         print(sample_commands)
        sample_commands = decode_commands(sample_commands)
        sample_insertions = target_insertions[:code_length,i].view(-1).tolist()
        sample_replacements = target_replacements[:code_length,i].view(-1).tolist()
        
        
        
        edits = (sample_commands, sample_insertions, sample_replacements)
        
        edited_code = perform_edits(code_to_edit, edits, gen_tok_id=stoi["<gen>"])
        
        edited_code_samples.append(edited_code)
        
    max_sample_len = max([len(sample) for sample in edited_code_samples])

    edited_code_samples = [(sample + max_sample_len * [stoi["<pad>"]])[:max_sample_len] for sample in edited_code_samples]
    return torch.tensor(edited_code_samples).T.to(device)
    
        
batch = next(iter(train_iterator))
decoder_input = batch.decoder_input
encoder_input = batch.encoder_input
ground_truth_code = batch.ground_truth_code
OOVss = batch.OOVs

gt_target_commands = batch.target_commands
gt_target_insertions = batch.target_insertions
gt_target_replacements = batch.target_replacements

gt_edited_code = outputs2code(gt_target_commands, gt_target_insertions, gt_target_replacements, decoder_input)

target_commands, target_insertions, target_replacements = model(encoder_input, decoder_input)

_, argmax_target_commands = target_commands.max(2)
_, argmax_target_insertions = target_insertions.max(2)
_, argmax_target_replacements = target_replacements.max(2)

outputs2code(argmax_target_commands, argmax_target_insertions, argmax_target_replacements, decoder_input)

for i in range(4):
    print("encoder_input      :", decode(rmpad(encoder_input[:,i].tolist()), []))
    print("ground_truth_code  :",decode(rmpad(ground_truth_code[:,i].tolist()), []))
    print("decoder_input      :",decode(rmpad(decoder_input[:,i].tolist()), []))
    print("gt_target_commands :", rmpad(gt_target_commands[:,i].tolist()))
    print("gt_target_insertions:", rmpad(gt_target_insertions[:,i].tolist()))
    print("gt_target_replacements:", decode(rmpad(gt_target_replacements[:,i].tolist()), []))
    print("gt_edited_code     :",decode(rmpad(gt_edited_code[:,i].tolist()), []))
    print()


encoder_input      : app_config , connection . alias and include_auto_created set to boolean True . call the method connection . creation . with arguments model , style and known_models , substitute the result for output and .
ground_truth_code  : <sos> output , = connection . creation . ( model , style , known_models )
decoder_input      : <sos> output , = connection . creation . ( model , style <gen> known_models )
gt_target_commands : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]
gt_target_insertions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
gt_target_replacements: ,
gt_edited_code     : <sos> output , = connection . creation . ( model , style , known_models )

encoder_input      : define the method with arguments self , name , , value , base , sysid , pubid and notation_name .
ground_truth_code  : <sos> def ( self , name , , value , base , sysid , pubid , notation_name ) :
decoder_input      : <sos> def ( self , name , <gen> <gen> value , base , sysid , pubid , not

### Retrieval baseline

In [33]:
BLEU_scores = []
for src, tgt in test_data:
    doc_ranking = retriever.BM25_search(src)
    if len(doc_ranking) > 0:
        top_2_doc = train_data[doc_ranking[0].doc]
        x_, y_ = top_2_doc
    else:
        x_, y_ = "", ""
    
    encoder_input, OOVs = encode_input(src)
    decoder_input = encode_output(tgt, OOVs)
    retrieved = encode_output(y_, OOVs)
    BLEU = nltk_bleu(decoder_input, retrieved)
    BLEU_scores.append(BLEU)
np.average(BLEU_scores)

0.4310125255226661

In [34]:
valid_iterator = BucketIterator(val_dataset,
    batch_size = 32,
    sort_key = lambda x: len(x.src)+len(x.tgt),
    device = device)

def batch_filter_ids(batch_list):
    return [[id for id in l if id not in [1,2,3]] for l in batch_list]

def evaluate(beam_size=1, log=False):
    model.eval() # Turn on the evaluation mode
    with torch.no_grad(), open("out.txt", "w") as out_fp:
        BLEU_scores = []
        ret_BLEU_scores = []
        
        for i, batch in enumerate(tqdm.tqdm(valid_iterator)):
            batch_size = batch.encoder_input.shape[1]
            max_iter = 8
            
            code_edits_finished = [None] * batch_size
            
            decoder_input = batch.decoder_input
            encoder_input = batch.encoder_input            
            OOVss = [[OOV_itos[OOV] for OOV in batch.OOVs.cpu()[:,idx].tolist() if OOV != 3] for idx in range(batch_size)]
            
            while not all(code_edits_finished) and max_iter > 0:
                max_iter -= 1
                target_commands, target_insertions, target_replacements = model(encoder_input, decoder_input)
                
                _, argmax_target_commands = target_commands.max(2)
                _, argmax_target_insertions = target_insertions.max(2)
                _, argmax_target_replacements = target_replacements.max(2)
                                
                code_samples = outputs2code(argmax_target_commands, argmax_target_insertions, argmax_target_replacements, decoder_input)
                                
                for j in range(batch_size):
                    if (stoi["<gen>"] not in code_samples[:,j] and code_edits_finished[j] == None) or max_iter == 0:
                        code_edits_finished[j] = rmpad(code_samples[:,j].cpu().tolist())
                        BLEU = nltk_bleu(
                            rmpad(batch.ground_truth_code[1:,j].cpu().tolist()),
                            code_edits_finished[j][1:])
                        retrieval_BLEU = nltk_bleu(
                            rmpad(batch.ground_truth_code[1:,j].cpu().tolist()),
                            rmpad(batch.decoder_input[1:,j].cpu().tolist()))
                        BLEU_scores.append(BLEU)
                        ret_BLEU_scores.append(retrieval_BLEU)
                        
                        out_fp.write(f"Decoder input: {decode(rmpad(batch.decoder_input[:,j].cpu().tolist()), OOVss[j])}\n")
                        out_fp.write(f"Target       : {decode(rmpad(batch.ground_truth_code[:,j].cpu().tolist()), OOVss[j])}\n")
                        out_fp.write(f"Prediction   : {decode(rmpad(code_samples[:,j].cpu().tolist()), OOVss[j])}\n")
                        out_fp.write(f"BLEU         : {BLEU:5.3f}\n")
                        out_fp.write(f"Ret BLEU     : {retrieval_BLEU:5.3f}\n")
                        out_fp.write("\n")
                
                decoder_input = code_samples[:max_seq_length]
                
            
            ground_truth_code = batch.ground_truth_code
                
        out_fp.write("\n\n| EVALUATION | BLEU: {:5.2f} | ret_BLEU: {:5.3f}\n".format(np.average(BLEU_scores), np.average(ret_BLEU_scores)))
        print("| EVALUATION | BLEU: {:5.3f} | ret_BLEU: {:5.3f}".format(np.average(BLEU_scores), np.average(ret_BLEU_scores)))

In [40]:
evaluate(beam_size=1)


| EVALUATION | BLEU: 0.431 | ret_BLEU: 0.431


In [44]:
def manual_eval(description, rertrieved_code):
    max_iter = 8
    model.eval()
    with torch.no_grad():
        encoder_input, OOVs = encode_input(description)
        encoder_input = torch.tensor(encoder_input).view(-1,1).to(device)
        
        decoder_input = encode_output(rertrieved_code, OOVs)
        decoder_input = [stoi["<sos>"]] + decoder_input
        decoder_input = torch.tensor(decoder_input).view(-1,1).to(device)
        
        while max_iter > 0:
            max_iter -= 1
            target_commands, target_insertions, target_replacements = model(encoder_input, decoder_input)
            
            _, argmax_target_commands = target_commands.max(2)
            _, argmax_target_insertions = target_insertions.max(2)
            _, argmax_target_replacements = target_replacements.max(2)

            code_samples = outputs2code(argmax_target_commands, argmax_target_insertions, argmax_target_replacements, decoder_input)
            
            print(decode(code_samples.view(-1).cpu().tolist(), OOVs))
            
            decoder_input = code_samples[:max_seq_length]

            
description = "if KeyError exception is caught,"
target = "except KeyError :"
rertrieved_code = "except foo :"

manual_eval(description, rertrieved_code)

<sos> if <unk> :
<sos> if <unk> :
<sos> if <unk> :
<sos> if <unk> :
<sos> if <unk> :
<sos> if <unk> :
<sos> if <unk> :
<sos> if <unk> :


In [35]:
criterion = nn.CrossEntropyLoss(ignore_index=stoi['<pad>'])
lr = 0.005 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.99)

In [38]:
def train_step(batch):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    vocab_size = len(itos) + max_seq_length
    encoder_input = batch.encoder_input
    decoder_input = batch.decoder_input
    ground_truth_commands = batch.target_commands
    ground_truth_insertions = batch.target_insertions
    ground_truth_replacements = batch.target_replacements

    optimizer.zero_grad()
    target_commands, target_insertions, target_replacements = model(encoder_input, decoder_input)

    
    command_loss = criterion(target_commands.view(-1, len(edit_stoi)), ground_truth_commands.view(-1))
    insertion_loss = criterion(target_insertions.view(-1, max_insertions+1), ground_truth_insertions.view(-1))
    replacement_loss = criterion(target_replacements.view(-1, vocab_size), ground_truth_replacements.view(-1))
    
    loss = command_loss + insertion_loss + replacement_loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    elapsed = time.time() - start_time
    return loss

In [ ]:
def train(steps=10000, log_interval=200, learning_interval=4000, eval_interval=1000):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    step = 1
    for batch in train_iterator:
        loss = train_step(batch)
        total_loss += loss.item()
        
        if step % log_interval == 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| {:5d}/{:5d} steps | '
                  'lr {:02.4f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    step, steps, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        
        if step % eval_interval == 0:
            print("Evaluating model")
            evaluate()
            model.train()
        
        if step % learning_interval == 0:
            scheduler.step()
        
        step += 1
        if step >= steps:
            print("Finished training")



            return

train(steps=1000000,eval_interval=10000,log_interval=400)

|   400/1000000 steps | lr 0.0050 | ms/batch 32.52 | loss  4.80 | ppl   121.50
|   800/1000000 steps | lr 0.0050 | ms/batch 32.32 | loss  4.66 | ppl   106.14
|  1200/1000000 steps | lr 0.0050 | ms/batch 33.16 | loss  4.56 | ppl    96.06
|  1600/1000000 steps | lr 0.0050 | ms/batch 32.39 | loss  4.48 | ppl    88.05
|  2000/1000000 steps | lr 0.0050 | ms/batch 32.81 | loss  4.43 | ppl    83.94
|  2400/1000000 steps | lr 0.0050 | ms/batch 32.27 | loss  4.39 | ppl    81.02
|  2800/1000000 steps | lr 0.0050 | ms/batch 32.39 | loss  4.33 | ppl    76.03
|  3200/1000000 steps | lr 0.0050 | ms/batch 32.49 | loss  4.32 | ppl    75.38
|  3600/1000000 steps | lr 0.0050 | ms/batch 32.37 | loss  4.28 | ppl    72.17
|  4000/1000000 steps | lr 0.0050 | ms/batch 32.23 | loss  4.28 | ppl    72.10
|  4400/1000000 steps | lr 0.0050 | ms/batch 32.40 | loss  4.22 | ppl    68.20
|  4800/1000000 steps | lr 0.0050 | ms/batch 32.37 | loss  4.22 | ppl    68.09
|  5200/1000000 steps | lr 0.0050 | ms/batch 32.13 |


| EVALUATION | BLEU: 0.444 | ret_BLEU: 0.430
| 10400/1000000 steps | lr 0.0049 | ms/batch 39.32 | loss  3.98 | ppl    53.64
| 10800/1000000 steps | lr 0.0049 | ms/batch 32.30 | loss  3.97 | ppl    53.04
| 11200/1000000 steps | lr 0.0049 | ms/batch 32.11 | loss  3.98 | ppl    53.46
| 11600/1000000 steps | lr 0.0049 | ms/batch 32.57 | loss  3.93 | ppl    50.70
| 12000/1000000 steps | lr 0.0049 | ms/batch 32.37 | loss  3.95 | ppl    51.71
| 12400/1000000 steps | lr 0.0049 | ms/batch 32.59 | loss  3.93 | ppl    50.98
| 12800/1000000 steps | lr 0.0049 | ms/batch 32.28 | loss  3.92 | ppl    50.47
| 13200/1000000 steps | lr 0.0049 | ms/batch 32.34 | loss  3.90 | ppl    49.44
| 13600/1000000 steps | lr 0.0049 | ms/batch 32.27 | loss  3.88 | ppl    48.43
| 14000/1000000 steps | lr 0.0049 | ms/batch 32.55 | loss  3.88 | ppl    48.52
| 14400/1000000 steps | lr 0.0049 | ms/batch 32.73 | loss  3.89 | ppl    48.90
| 14800/1000000 steps | lr 0.0049 | ms/batch 32.56 | loss  3.86 | ppl    47.29
| 1520


| EVALUATION | BLEU: 0.431 | ret_BLEU: 0.432
| 20400/1000000 steps | lr 0.0048 | ms/batch 40.09 | loss  3.76 | ppl    42.99
| 20800/1000000 steps | lr 0.0048 | ms/batch 32.45 | loss  3.75 | ppl    42.61
| 21200/1000000 steps | lr 0.0048 | ms/batch 32.24 | loss  3.73 | ppl    41.78
| 21600/1000000 steps | lr 0.0048 | ms/batch 32.59 | loss  3.74 | ppl    42.22
| 22000/1000000 steps | lr 0.0048 | ms/batch 32.35 | loss  3.74 | ppl    42.07
| 22400/1000000 steps | lr 0.0048 | ms/batch 32.35 | loss  3.73 | ppl    41.76
| 22800/1000000 steps | lr 0.0048 | ms/batch 32.68 | loss  3.73 | ppl    41.51
| 23200/1000000 steps | lr 0.0048 | ms/batch 32.54 | loss  3.72 | ppl    41.20
| 23600/1000000 steps | lr 0.0048 | ms/batch 32.30 | loss  3.69 | ppl    40.12
| 24000/1000000 steps | lr 0.0048 | ms/batch 32.48 | loss  3.72 | ppl    41.30
| 24400/1000000 steps | lr 0.0047 | ms/batch 32.57 | loss  3.70 | ppl    40.46
| 24800/1000000 steps | lr 0.0047 | ms/batch 32.46 | loss  3.70 | ppl    40.26
| 2520

# Evaluate